In [14]:
import pandas as pd
import requests
import json

In [15]:
# Define the API endpoint
#type = [week, industry]
#sessionID = 
#abstractAPI = 
#dayID = (from 2 until gets nothin)
#presentationID = 

#daysurl = https://ueg2024.planner.documedias.systems/api/program/rooms/2?program_type={$type}&system=program_week_2024&program_mode=list&program_sort=date&filter_display_type=1,3&filter_group=is_not_child
sessionurl = "https://ueg2024.planner.documedias.systems/api/program/sessions/80?program_type=week&system=program_week_2024"
#abstracturl = https://ueg2024.planner.documedias.systems/api/program/presentations/2506?program_type=week&system=program_week_2024

# Make the API request
responseSession = requests.get(sessionurl)
dataSession = responseSession.json()

# ITERATE OVER EACH FIELD

In [22]:
import json

# Assuming you've already loaded keywords_data from the JSON file
with open('keywords.json', 'r') as f:
    keywords_data = json.load(f)

# Define the lists
indication = []
asset = []
company_name = []
MoA = []
other_keywords = []

# Populate the lists from the keywords dictionary
for row in keywords_data['rows']:
    indication.append(row[0])
    asset.append(row[1])
    company_name.append(row[2])
    MoA.append(row[3])
    other_keywords.append(row[4])

# Function to check if a keyword is present in the text
def find_matching_keyword(text, keywords):
    if not text:
        return None
    for keyword in keywords:
        if keyword and keyword.lower() in text.lower():
            return keyword
    return None

# Print the lists to verify
print("Indication:", indication)
print("Asset:", asset)
print("Company Name:", company_name)
print("MoA:", MoA)
print("Other Keywords:", other_keywords)


Indication: ['Crohn', "Crohn's disease", 'IBD', 'inflammatory bowel disease', 'Ulcerative Colitis', 'Ulcerative', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
Asset: ['Tulisokibart', 'Ustekinumab', 'Guselkumab', 'Upadacitinib', 'Risankizumab', 'Mirikizumab', 'Ozanimod', 'Etrasimod', 'CT-P13', 'ABX-464', 'MORF-057', 'VTX958', 'TEV-48574', 'Humira', 'PF-07054894', 'PF-07261271', 'Ritlecitinib', 'PF-06480605', 'Infliximab', 'Cimzia', 'Golimumab', 'Vedolizumab', 'SHR0302', 'Neihulizumab', 'TAK-279', 'Tysabri', 'AGMB-129']
Company Name: ['Merck', 'JnJ', 'Prometheus', 'AbbVie', 'Eli Lilly', 'BMS', 'Pfizer', 'Celltrion', 'Abivax', 'Morphic', 'Ventyx', 'Sanofi', 'UCB', 'Takeda', 'Reistone', 'Altru Bio', 'Biogen', 'Agomab', 'Boehringer', 'Incannex', 'Medibiofarma', 'EA Pharma', 'Gilead', 'Spyre', 'Mozart', 'Athos', 'Syneos']
MoA: ['TL1A', 'Anti-IL-12/23', 'Interleukin 23 antagonist', 'JAK inhibitor', 'S1PR modulators', 'Anti TNF', 'MiRNA-124 expression sti

In [23]:

# Define the fields you want to extract for sessions and presentations combined
fields = [
    "Session Date", "Session Start Time", "Session End Time", "Session Timezone",
    "Session Location", "Session Number", "Session Pathways", "Session Format",
    "Session CME", "Session Type", "Session Group", "Session Title", "Session Description",
    "Presentation Date", "Presentation Location", "Presentation Start Time", "Presentation End Time",
    "Presentation ID", "Presentation Number", "Presentation Title", "Abstract Authors", "Abstract Affiliations", "Presentation Title (lowercase)",
    "Abstract", "Abstract Url", "Company", "Asset", "Indication", "MoA", "Other Keywords"
]

# Initialize the combined data list
combined_data = []



# Iterate over each session item
for item in dataSession:
    # Common session-related data (outside presentation loop)
    session_common_data = {
        "Session Date": item.get('day', {}).get("date"),
        "Session Start Time": item.get("start_time"),
        "Session End Time": item.get("end_time"),
        "Session Timezone": "CEST",
        "Session Location": item.get("room", {}).get('short_name'),
        "Session Number": item.get("id"),
        "Session Pathways": ', '.join([pathway.get('name') for pathway in item.get('pathways', [])]),
        "Session CME": "",
        "Session Type": item.get('session_type', {}).get('name'),
        "Session Group": item.get("session_group_id"),
        "Session Title": item.get("title"),
        "Session Description": item.get("content", {}).get("outline"),
    }

    for presentation in item.get('presentations', []):
        
        abstracts = {}
        Abstract_url = None
        Abstract_id = presentation.get("presentation", {}).get("abstract_id")
        if (Abstract_id):
            Abstract_url = f"https://programme.ueg.eu/week2024/#/details/presentations/{Abstract_id}"
            Abstract_endpoint = f"https://ueg2024.abstract.documedias.systems/api/v1/manager/abstract/multi/html/id/{Abstract_id}/template/planner_preview?program_type=week&system=program_week_2024"
            responseAbstract = requests.get(Abstract_endpoint)
            dataAbstract = responseAbstract.json()

            abstracts = dataAbstract.get(f"{Abstract_id}")
            print(abstracts)

        authors = []
        institutions = []

        for person in presentation.get('presentation', {}).get('persons', []):
            #print(person)
            institution = person.get('person', {}).get('institution')
            author_name = person.get('person', {}).get('first_name') + ' ' + person.get('person', {}).get('last_name')
            if institution:
                institutions.append(institution)
                #print(institutions)
            if author_name:
                authors.append(author_name)

            presentation_title = presentation.get("presentation", {}).get("title", "")
            session_title = item.get("title", "")
            session_description = item.get("content", {}).get("outline", "")

            matched_indication = find_matching_keyword(presentation_title, indication) or \
                                find_matching_keyword(session_title, indication) or \
                                find_matching_keyword(session_description, indication)

            matched_asset = find_matching_keyword(presentation_title, asset) or \
                            find_matching_keyword(session_title, asset) or \
                            find_matching_keyword(session_description, asset)

            matched_company = find_matching_keyword(presentation_title, company_name) or \
                            find_matching_keyword(session_title, company_name) or \
                            find_matching_keyword(session_description, company_name)

            matched_MoA = find_matching_keyword(presentation_title, MoA) or \
                        find_matching_keyword(session_title, MoA) or \
                        find_matching_keyword(session_description, MoA)

            matched_other_keywords = find_matching_keyword(presentation_title, other_keywords) or \
                                    find_matching_keyword(session_title, other_keywords) or \
                                    find_matching_keyword(session_description, other_keywords)


        presentation_data = {
            **session_common_data,  # Include session data in each presentation entry
            "Session Format": ', '.join([format.get('name') for format in item.get('formats', [])]),
            "Presentation Date": item.get('day', {}).get("date"),
            "Presentation Location": item.get("room", {}).get('short_name'),
            "Presentation Start Time": presentation.get("start_time"),
            "Presentation End Time": presentation.get("end_time"),
            "Presentation ID": presentation.get("id"),
            "Presentation Number": presentation.get("code"),
            "Presentation Title": presentation.get("presentation", {}).get("title"),
            "Presentation Title (lowercase)": presentation.get("presentation", {}).get("title", '').lower(),
            "Abstract Authors":', '.join(authors),
            "Abstract Affiliations": ', '.join(institutions),  # Join institutions into a string
            "Abstract": abstracts,  # IF ABSTRACT_ID:
            "Abstract Url": Abstract_url,
            "Company": matched_company,
            "Asset": matched_asset,
            "Indication": matched_indication,
            "MoA": matched_MoA,
            "Other Keywords": matched_other_keywords,
        }



        #print(author_name)
        print(combined_data)
        # Append combined data
        combined_data.append(presentation_data)

<i><u>B.E. Sands</u><sup>1</sup>, G. D’Haens<sup>2</sup>, S. Danese<sup>3</sup>, T. Hisamatsu<sup>4</sup>, W. Reinisch<sup>5</sup>, N.A. Terry<sup>6</sup>, L. Salese<sup>6</sup>, R. Van Rampelbergh<sup>7</sup>, Z. Yang<sup>6</sup>, J. Johanns<sup>6</sup>, G. DuVall<sup>8</sup>, N. Abu Farsakh<sup>9</sup>, R. Panaccione<sup>10</sup></i><br>
<div><sup>1</sup>Dr. Henry D Janowitz Division of Gastroenterology, Icahn School of Medicine at Mount Sinai, New York, United States of America, <sup>2</sup>Department of Gastroenterology, Amsterdam University Medical Centers, Amsterdam, Netherlands, <sup>3</sup>Gastroenterology and Endoscopy, IRCCS San Raffaele Hospital and Vita-Salute San Raffaele University, Milan, Italy, <sup>4</sup>Department of Gastroenterology and Hepatology, Kyorin University, Tokyo, Japan, <sup>5</sup>Division of Gastroenterology & Hepatology, Medical University of Vienna, Vienna, Austria, <sup>6</sup>Janssen Research & Development, LLC, Spring House, United States of Americ

In [24]:
# Create a DataFrame from the combined data list
df_combined = pd.DataFrame(combined_data, columns=fields)

# Export the DataFrame to an Excel file
df_combined.to_excel("UEG_Scrape_Combined.xlsx", sheet_name="Combined Data", index=False)

# Display the DataFrame (for debugging purposes)
print(df_combined)

  Session Date Session Start Time Session End Time Session Timezone  \
0   2024-10-14              08:30            09:30             CEST   
1   2024-10-14              08:30            09:30             CEST   
2   2024-10-14              08:30            09:30             CEST   
3   2024-10-14              08:30            09:30             CEST   
4   2024-10-14              08:30            09:30             CEST   

  Session Location  Session Number Session Pathways        Session Format  \
0               A3              80              IBD  Live, Recording, Q&A   
1               A3              80              IBD  Live, Recording, Q&A   
2               A3              80              IBD  Live, Recording, Q&A   
3               A3              80              IBD  Live, Recording, Q&A   
4               A3              80              IBD  Live, Recording, Q&A   

  Session CME      Session Type  ...           Abstract Authors  \
0      Pfizer  Abstract Session  ...       